I've been concerned that previous representations use 4 states for the quaternion representation of the orientation.  Despite all of the issues of using Euler angles, there is a desire to use them because what we really care about is the uncertainty of the Euler angle states.

This derivation uses Euler angles (heading, roll, pitch) as Kalman states but uses quaternions for the underlying calculations.

In [3]:
from sympy import *
import numpy as np

| State Var | Description            | Units   |
|:-----------|:------------------------|:---------:|
| $heading$      | Heading (rotation about z axis)    | rad  |
| $pitch$      | Rotation about y axis    | rad |
| $roll$      | Rotation about x axis     | rad |
| $a_x$     | Body axis acceleration | m/s$^2$ |
| $a_y$     | Body axis acceleration | m/s$^2$ |
| $a_z$     | Body axis acceleration | m/s$^2$ |
| $w_x$     | Body axis rotation rate | rad/s |
| $w_y$     | Body axis rotation rate | rad/s |
| $w_z$     | Body axis rotation rate | rad/s |

To convert from inertial to body coordinate system:
$$
v_{body} = q^{-1} v_{inertial} q
$$
and to convert from body to inertial coordinate system:
$$
v_{inertial} = q v_{body} q^{-1}
$$

## Coordinate system

### Global Coordinate System

Use NED (North-East-Down coordinate system)

x points in direction of North

y points in the direction of East

z points in the direction of down

### Aircraft Coordinate System

x points in the lognitudinal axis -- the direction of travel

y points from the center of the vehicle out along the right wing

z points down

In [30]:
def euler2q(heading, pitch, roll):
    q = Quaternion.from_axis_angle([0, 0, 1], heading)
    q = q * Quaternion.from_axis_angle([0, 1, 0], pitch)
    q = q * Quaternion.from_axis_angle([1, 0, 0], roll)
    return q


def q2roll(q):
    return atan2(2*(q.a*q.b + q.c*q.d),
                 1-2*(q.b*q.b + q.c*q.c))


def q2heading(q):
    return atan2(2*(q.a*q.d + q.b*q.c),
               1-2*(q.c*q.c + q.d*q.d))


def q2pitch(q):
    return asin(2*(q.a*q.c - q.d*q.b))


def q2np(q):
    return np.array([q.a, q.b, q.c, q.d])


def simplify_cse(F, outputmode='ccode'):
    # Simplify with common expressions

    shape = F.shape
    flat_len = len(F.flatten())
    subexprs, subs = cse(F.flatten())
    #subs = np.reshape(subs, shape)

    for se in subexprs:
        if outputmode == 'python':
            print(f"{se[0]} = {se[1]}")
        else:
            print(f"float {se[0]} = {ccode(se[1])};")

    for i in range(flat_len):
        idx = np.unravel_index(i, shape)
        idx_str = f"{idx[0]}, {idx[1]}" if len(idx) > 1 else f"{idx[0]}"
        tmp = subs[i]
        if tmp != 0:
            if outputmode == 'python':
                print(f"F[{idx_str}] = {tmp}")
            else:
                print(f"F({idx_str}) = {ccode(tmp)};")

# Kalman Prediction

The model used for this filter is an assumption of coordinated flight.  Using this assumption, we can calculate the [rate of rotation](https://en.wikipedia.org/wiki/Standard_rate_turn) (about inertial z axis) and [centripital acceleration](https://en.wikipedia.org/wiki/Centripetal_force) due to this rotation.  

Centripital acceleration derivation by considering the triangle of a banked airplane with one leg of centripital acceleration and the other gravity.
$$ \tan \phi = \frac{a_c}{g} $$
where $\phi$ is the angle of bank.  $a_c=v^2/r$ so this equation can also be expressed as:
$$ \tan \phi = \frac{v^2}{r g} $$

Rate of rotation derivation. The aircraft completes a full rotation of a circle of radius $r$ in $2\pi r/v$ seconds.  Therefore, the rate of rotation (in inertial coordinates) can be written as:
$$ \dot{\psi} = \frac{2 \pi v}{2\pi r} = \frac{v}{r} = a_c/v $$
The rate of rotation simplifies to:
$$ \dot{\psi} = \frac{g}{v} \tan \phi $$

In [16]:
heading, pitch, roll = symbols(['heading', 'pitch', 'roll'])
q0, q1, q2, q3 = symbols(['q0', 'q1', 'q2', 'q3'])
wx, wy, wz = symbols(['wx', 'wy', 'wz'])
wbx, wby, wbz = symbols(['wbx', 'wby', 'wbz'])
abx, aby, abz = symbols(['abx', 'aby', 'abz'])
ax, ay, az = symbols(['ax', 'ay', 'az'])
mx, my, mz = symbols(['mx', 'my', 'mz'])
tas = symbols('tas')
state = [heading, pitch, roll, ax, ay, az, wx, wy, wz, wbx, wby, wbz, abx, aby, abz]

G = symbols('G')
dt = symbols('dt')

In [37]:
q = euler2q(heading, pitch, roll)

qdelta = Quaternion(1, wx/2*dt, wy/2*dt, wz/2*dt)
qnext = q * qdelta
qnext = qnext.normalize()

# vector, in inertial coords, in direction which ac acts (along right wing (0,1,0) for positive phi)
v_ac = q2np(q * Quaternion(0, 0, 1, 0) * q.inverse())[1:]
v_ac[2] = 0
v_ac /= sqrt(v_ac[0]**2 + v_ac[1]**2)

ac = v_ac*G*tan(roll)
ag = np.array([0,0,-G])  
a = ag + ac

ab = q2np(q.inverse() * Quaternion(0, *a) * q)[1:]

wpredict = q2np(q.inverse() * Quaternion(0, 0, 0, G / tas * tan(roll)) * q)[1:]

state_next = [q2heading(qnext), q2pitch(qnext), q2roll(qnext)] + ab.tolist() + wpredict.tolist() + [wbx, wby, wbz, abx, aby, abz]


## Prediction Jacobian

$F$ is all the partial derivatives of the equations used to predict the next state.

In [38]:
F = np.zeros((len(state), len(state)), dtype=object)
for n in range(len(state)):
    for m in range(len(state)):
        tmp = diff(state_next[n], state[m])  
        F[n, m] = tmp
        
simplify_cse(F)

float x0 = (1.0/2.0)*roll;
float x1 = sin(x0);
float x2 = (1.0/2.0)*heading;
float x3 = sin(x2);
float x4 = (1.0/2.0)*pitch;
float x5 = sin(x4);
float x6 = x3*x5;
float x7 = x1*x6;
float x8 = (1.0/2.0)*x7;
float x9 = cos(x0);
float x10 = cos(x2);
float x11 = cos(x4);
float x12 = x10*x11;
float x13 = x12*x9;
float x14 = (1.0/2.0)*x13;
float x15 = x14 + x8;
float x16 = dt*wx;
float x17 = x15*x16;
float x18 = x11*x3;
float x19 = x18*x9;
float x20 = (1.0/2.0)*x19;
float x21 = -x20;
float x22 = x10*x5;
float x23 = x1*x22;
float x24 = (1.0/2.0)*x23;
float x25 = x21 + x24;
float x26 = dt*wy;
float x27 = x25*x26;
float x28 = x1*x18;
float x29 = (1.0/2.0)*x28;
float x30 = x22*x9;
float x31 = (1.0/2.0)*x30;
float x32 = x29 + x31;
float x33 = dt*wz;
float x34 = x32*x33;
float x35 = x1*x12;
float x36 = x6*x9;
float x37 = -x36;
float x38 = x35 + x37;
float x39 = -x23;
float x40 = x19 + x39;
float x41 = (1.0/2.0)*x16;
float x42 = x13 + x7;
float x43 = (1.0/2.0)*x26;
float x44 = -x35;
float x45 = x36

F(2, 7) = x168*(x144*x163 + x145 + x172*x40) + x169*(-x144*x173 - x144*x174 + x161 + x162*x48);
F(2, 8) = x168*(x149*x163 + x150 - x172*x47) + x169*(-x136 + x138 + x143 - x149*x173 - x149*x174);
F(3, 0) = x122*x238 + x15*x228 + x219*x42 + x225*x38 + x227*x40 + x231*x25 + x241;
F(3, 1) = x122*x273 + x228*x90 + x231*x92 + x239*x242 + x240*x98 + x263*x40 + x271*x42 + x272*x38;
F(3, 2) = x112*x231 + x122*x305 + x228*x57 + x297*x42 + x301*x38 + x302*x40 + x306;
F(4, 0) = x15*x307 + x225*x47 + x227*x42 + x228*x25 + x238*x38 + x239*x57 + x240*x63 + x40*(x175 - x203 + x210 - x216 + x218 + x304);
F(4, 1) = x102*x240 + x228*x92 + x239*x98 + x263*x42 + x272*x47 + x273*x38 + x307*x90 + x40*(x264 - x265 + x266 - x267 + x268 - x269 + x270);
F(4, 2) = x112*x228 + x119*x240 + x15*x239 + x301*x47 + x302*x42 + x305*x38 + x307*x57 + x40*(x177 + x237 - x276 + x278 - x284 + x292 - x294 + x296);
F(5, 0) = x219*x47 + x225*x40 + x227*x45 + x228*x32 + x231*x63 + x238*x42 + x306;
F(5, 1) = x102*x231 + x228*x94 

## Accel update

The Kalman innovation is $y = z - h(x)$ and $H$ matrix is the partial derivatives of the $h(x)$ function.  Since accelerations are states in the filter, $h(x)$ is trivial.

In [33]:
haccel = np.array([ax + abx, ay + aby, az + abz])
H = np.zeros((3, len(state)), dtype=object)
for n in range(3):
    for m in range(len(state)):
        tmp = diff(haccel[n], state[m])
        H[n, m] = tmp
        if tmp != 0:
            print(f"H[{n}, {m}] = {tmp}")

H[0, 3] = 1
H[0, 12] = 1
H[1, 4] = 1
H[1, 13] = 1
H[2, 5] = 1
H[2, 14] = 1


## Gyro update

Similarly for the gyro, body rotation rates are part of the state so the $H$ matrix is trivial.

In [34]:
hgyro = np.array([wx + wbx, wy + wby, wz + wbz])
H = np.zeros((3, len(state)), dtype=object)
for n in range(3):
    for m in range(len(state)):
        tmp = diff(hgyro[n], state[m])
        H[n, m] = tmp
        if tmp != 0:
            print(f"H[{n}, {m}] = {tmp}")

H[0, 6] = 1
H[0, 9] = 1
H[1, 7] = 1
H[1, 10] = 1
H[2, 8] = 1
H[2, 11] = 1


## GPS Bearing Update

y = meas - h(x)

In [35]:
#meas = np.array([cos(gps_heading), sin(gps_heading)])
h = np.array([cos(heading), sin(heading)])

for n in range(2):
    for m in range(len(state)):
        tmp = diff(h[n], state[m])
        if tmp != 0:
            print(f"H[{n}, {m}] = {tmp}")

H[0, 0] = -sin(heading)
H[1, 0] = cos(heading)


# Test Cases

## Standard rate turn


In [47]:
tas = 92.6  # m/s^2  => close to 180 knots
roll = 25*np.pi/180
g = 9.81

turn_rate = g / tas * tan(roll)  # 1/s
print(2*np.pi / turn_rate)
print(127 / .02)

q = Quaternion.from_axis_angle([1, 0, 0], roll)
w = q2np(q.inverse() * Quaternion(0, 0, 0, turn_rate) * q)[1:]
print(w)

v_ac = q2np(q * Quaternion(0, 0, 1, 0) * q.inverse())[1:]
v_ac[2] = 0
v_ac /= sqrt(v_ac[0]**2 + v_ac[1]**2)

ac = v_ac*g*tan(roll)
ag = np.array([0,0,-g])  
a = ag + ac
ab = q2np(q.inverse() * Quaternion(0, *a) * q)[1:]
print(ab)

127.188925896105
6350.0
[0 0.0208775161359882 0.0447719778366767]
[0 -4.44089209850063e-16 -10.8241373850220]
